In [45]:
import numpy as np
import pandas as pd

from scipy.ndimage.interpolation import shift

In [46]:
%%time
# df con los datos de las tendencias
df = pd.read_csv("C:/Users/Daniel/Desktop/por hora/tweets_24_notedencias_preprocesado_labels_hora.csv", sep = ";")
df = df.reset_index(drop = True)

# df que tiene la hora de inicio de las tendencias
df_start_lifetime = pd.read_csv("C:/Users/Daniel/Desktop/por hora/tweets_24_start_lifetime_notendencias_hora.csv", sep = ";")

<string>:2: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.


Wall time: 24.6 s


In [47]:
df["trend"] = df.trends.apply(lambda x : x)
df.drop("trends", axis = 1, inplace = True)

In [48]:
# df_summary
# Es el df donde vamos a guardar la informacion de los tweets
# Primero agregamos los trends del dataframe df 
# Segundo agregamos la hora donde empieza a ser tendencia (merge)
# Vamos a estudiar el comportamiento de las tendencias 6 horas antes de que lo sean
# Por lo tanto vamos a quitar las tendencias entre las horas 0 y 5 ya que no tienen 6 horas de informacion

df_summary = df_start_lifetime.copy()
#df_summary["trend"] = df.trend.unique()

#df_summary = pd.merge(left = df_summary, right = df_start_lifetime, on = "trend", how = "inner")

#df_summary.start_lifetime = df_summary.start_lifetime.apply(lambda x : int(x[:2]))

#df_summary = df_summary[df_summary.start_lifetime >= 6]

#df_summary = df_summary.reset_index(drop = True)

In [51]:
df_summary.start_lifetime.value_counts()

23    49
22    49
7     49
8     49
9     49
10    49
11    49
12    49
13    49
14    49
15    49
19    49
20    49
21    49
6     49
16    48
17    48
18    48
Name: start_lifetime, dtype: int64

In [ ]:
# Me quedo con las tendencias que comiencen a las 06:00 o más tarde
#df = df[df.trend.isin(df_summary.trend.unique())]

In [53]:
%%time

starts = [i for i in range(24)]

to_df = list()
for start in starts:
    # Creo una lista con las horas desde 6 horas antes de start
    hours = [start - i for i in range(6, -1, -1)]
    
    # Filtro el df_summary por las tendencias que comiencen en la hora "start"
    df_aux = df_summary[df_summary.start_lifetime == start]
    df_aux = df_aux.reset_index(drop = True)
    
    list_aux = list()
    for trend in df_aux.trend:
        
        lista_filtro = list()
        
        # Calcula todas las variables de agregacion
        df_filtro = df[(df.trend == trend) & (df.hour.isin(hours))]
        
        total_tweets      = df_filtro.shape[0]

        total_hashtags    = df_filtro.hashtags_count.sum()

        total_mentions    = df_filtro.mentions_count.sum()

        total_reply_to    = df_filtro.reply_to_count.sum()

        total_url         = df_filtro.urls.sum()

        total_photo       = df_filtro.photos.sum()

        total_retweets    = df_filtro.retweets_count.sum()

        total_likes       = df_filtro.likes_count.sum()

        total_replies     = df_filtro.replies_count.sum()

        total_interaction = df_filtro.interaccion.sum()
        
        lista_filtro.extend([total_tweets, total_hashtags, total_mentions, total_reply_to, total_url,
                             total_photo, total_retweets, total_likes, total_replies, total_interaction])
        
        # Crea las variables dependientes del tiempo
        tweet_counts = [df[(df.trend == trend) & (df.hour == hour)].shape[0] for hour in hours]

        tweet_counts = list(tweet_counts + shift(tweet_counts, 1))
        vel          = list(tweet_counts - shift(tweet_counts, 1))
        acc          = list(vel          - shift(vel         , 1))
        
        user_count = [df[(df.trend == trend) & (df.hour == hour)].username.unique().shape[0] for hour in hours]
        user_count = list(user_count + shift(user_count, 1))

        
        lista_filtro.extend(tweet_counts + vel + acc + user_count)
        
        list_aux.append(lista_filtro)
        
    to_df.extend(list_aux)
    print(start)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
Wall time: 1h 10min 16s


In [55]:
columns_labels_agg = ["total_tweets", "total_hashtags", "total_mentions", "total_reply_to", "total_url",
                      "total_photo", "total_retweets", "total_likes", "total_replies", "total_interaction"]

In [56]:
count_labels      = ["tweet_count_{}{}".format(i, i + 1) for i in range(7)]
vel_labels        = ["tweet_vel_{}{}".format(i, i + 1)   for i in range(7)]
acc_labels        = ["tweet_acc_{}{}".format(i, i + 1)   for i in range(7)]
user_count_labels = ["user_count_{}{}".format(i, i + 1)  for i in range(7)]

columns_labels_time = count_labels + vel_labels + acc_labels + user_count_labels

In [57]:
columns_labels = columns_labels_agg + columns_labels_time

In [58]:
df_features = pd.DataFrame(to_df, columns = columns_labels)

In [59]:
df_summary = pd.concat([df_summary, df_features], axis = 1)

In [60]:
df_summary

,trend,start_lifetime,total_tweets,total_hashtags,total_mentions,total_reply_to,total_url,total_photo,total_retweets,total_likes,...,tweet_acc_45,tweet_acc_56,tweet_acc_67,user_count_01,user_count_12,user_count_23,user_count_34,user_count_45,user_count_56,user_count_67
0,puedo,6,2,2,2,2,0,1,3.0,174.0,...,0,0,2,0,0,0,0,0,0,2
1,agradecida,6,2,2,2,2,0,0,6.0,76.0,...,0,0,2,0,0,0,0,0,0,1
2,siempre,6,6,6,6,6,0,1,3.0,46.0,...,0,0,6,0,0,0,0,0,0,3
3,gente,6,2,2,2,2,0,0,3.0,42.0,...,0,0,2,0,0,0,0,0,0,2
4,buenos,6,4,4,10,4,0,2,1.0,38.0,...,0,0,4,0,0,0,0,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
874,decir,23,8382,8382,8687,12947,224,417,14527.0,61686.0,...,18,47,6,1180,2322,2215,2205,2215,2258,2318
875,historia,23,5215,5215,5445,7430,659,675,12383.0,53470.0,...,59,199,-29,712,1494,1471,1330,1239,1331,1408
876,puedo,23,5925,5925,6030,7498,133,358,20467.0,100796.0,...,118,27,-42,801,1636,1602,1537,1561,1631,1695
877,vamos,23,5696,5696,5975,7912,314,528,14242.0,54549.0,...,73,64,21,747,1532,1513,1457,1454,1520,1627


In [ ]:
df_summary.to_csv("tweets_24_notendencias_variables_hora.csv", sep = ";", index = False)

In [ ]:
#df_summary.iloc[:, :2].to_csv("summary_25.csv", sep = ";", index = False)

In [ ]:
df_summary.shape